In [11]:
import networkx as nx
import pygraphviz

In [39]:
#path_to_dag = '/home/makrai/project/hypernym18-SemEval/top500words.dot'
path_to_dag = '1A_UMBC_tokenized.txt_100_cbow.vec.gz_True_200_0.3_unit_True_vocabulary_filtered.alph.reduced2_more_permissive.dot'
dag = nx.drawing.nx_agraph.read_dot(path_to_dag)

In [40]:
#nx.write_gpickle(dag, '/home/makrai/project/hypernym18-SemEval/top500words.gpickle')

In [41]:
deepest_occurrence = {}
attributes_for_nodes = {}
for n in dag.nodes(data=True):
    words = n[1]['label'].split('|')[1].split('\\n')
    node_id = int(n[1]['label'].split('|')[0])
    attributes_for_nodes[node_id] = n[1]['label'].split('|')[2].split('\\n')
    for w in words:
        if not w in deepest_occurrence or deepest_occurrence[w][1] > len(words):
            deepest_occurrence[w] = (node_id, len(words))

In [42]:
print(deepest_occurrence['dog'])
print(attributes_for_nodes[deepest_occurrence['dog'][0]])
print(deepest_occurrence['poodle'])
print(attributes_for_nodes[deepest_occurrence['poodle'][0]])

(64964, 2)
['n198', 'n86', 'n89', 'n167', 'n180']
(47845, 12)
['n198', 'n80', 'n167', 'n180']


In [43]:
dog_location = deepest_occurrence['dog'][0]
poodle_location = deepest_occurrence['poodle'][0]
all_paths = nx.all_simple_paths(dag,'node{}'.format(dog_location), 'node{}'.format(poodle_location))
print(list(all_paths))

[]


In [44]:
def generate_file_names(dataset_dir, dataset_id):
    dataset_mapping = {
        '1A':['english', 'UMBC'],
        '1B':['italian', 'it_itwac'],
        '1C':['spanish', 'es_1Billion'],
        '2A':['medical', 'med_pubmed'],
        '2B':['music', 'music_bioreviews']
    }
    data_file = '{}/training/data/{}.{}.training.data.txt'.format(
        dataset_dir,
        dataset_id,
        dataset_mapping[dataset_id][0]
    )
    gold_file = '{}/training/gold/{}.{}.training.gold.txt'.format(
        dataset_dir,
        dataset_id,
        dataset_mapping[dataset_id][0]
    )
    vocab_file='{}/vocabulary/{}.{}.vocabulary.txt'.format(
        dataset_dir,
        dataset_id,
        dataset_mapping[dataset_id][0]
    )
    return data_file, gold_file, vocab_file

In [45]:
dataset_dir = '/home/berend/datasets/semeval2018/SemEval18-Task9'
dataset_id = path_to_dag[0:2]
train_data_file, train_gold_file, vocab = generate_file_names(dataset_dir, dataset_id)
dev_data_file, dev_gold_file = train_data_file.replace('training', 'trial'), train_gold_file.replace('training', 'trial')
test_data_file = train_data_file.replace('training', 'test')

train_queries = [l.split('\t')[0].replace(' ', '_') for l in open(train_data_file)] # do we want to consider category as well?
train_golds = [
    [x.replace(' ', '_') for x in line.strip().split('\t')] for line in open(train_gold_file)
]

dev_queries = [l.split('\t')[0].replace(' ', '_') for l in open(dev_data_file)]
dev_golds = [
    [x.replace(' ', '_') for x in line.strip().split('\t')] for line in open(dev_gold_file)
]

test_queries = [l.split('\t')[0].replace(' ', '_') for l in open(test_data_file)]

In [46]:
dag_reversed = dag.reverse()
shortest_path_lengths = []
hypernym_pairs_included = []
paths = []
for query, hypernyms in zip(train_queries, train_golds):
    if query in deepest_occurrence:
        query_location = deepest_occurrence[query][0]
        for gold in hypernyms:
            if not gold in deepest_occurrence:
                continue
            hypernym_pairs_included.append((query, gold))
            gold_location = deepest_occurrence[gold][0]
            if gold_location == query_location:
                shortest_path_lengths.append(0)
                paths.append([query_location])
            else:
                all_paths = list(nx.all_simple_paths(dag_reversed, 'node{}'.format(query_location), 'node{}'.format(gold_location)))
                paths.append(all_paths)
                if len(all_paths) == 0:
                    shortest_path_lengths.append(-1)
                else:
                    shortest_path_lengths.append(min([len(p)-1 for p in all_paths]))

In [47]:
from collections import Counter
print(Counter([spl for spl in shortest_path_lengths]))

Counter({-1: 9977, 1: 202, 0: 184, 2: 183, 3: 85, 4: 7, 5: 2})
